# WLASL (World Level American Sign Language)

I am using a dataset that I downloaded from Kaggle. This dataset has 2,000 words, and over 20,000 videos of these 2,000 words in total.

## JSON Parsing and Organizing Videos/Frames

The video names represent the video-id of that video. To find out what word the video is showing, I have to find the video ID in the WLASL_v0.3.json file and see what word that video ID belongs to. The structure of the json file is as follows:

```
[
    {
        "gloss": "<word>",
        "instances": [
            {
                ...,
                ...,
                ...,
                "video_id": "<video_id_num>"
            },
            {
                ...,
                ...,
                ...,
                "video_id": "video_id_num"
            },
            ...
        ]
    },
    {
        "gloss": "<word>",
        "instances": [
            {
                ...,
                ...,
                ...,
                "video_id": "<video_id_num>"
            },
            ...
        ]
    },
    ...
]
```

To read and parse through json files, there is a json module that is downloaded alongside python. I will use this to parse through the json file and organize all the videos into their own directory in one of the three splits: Train, Test, or Validation. In addition, each video will be split into individual frames and the frames will be saved as png images. The final directory structure will look like:
```
/Videos
|--<split_type>
|   |--<word_id>
|   |   |--<instance_id>
|   |   |   |--<instance_id>_<frame_num>.png
|   |   |   |--<instance_id>_<frame_num>.png
|   |   |   ...
|   |   |--<instance_id>
|   |   |   |--<instance_id>_<frame_num>.png
|   |   |   |--<instance_id>_<frame_num>.png
|   |   |   ...
|   |   ...
|   |--<word_id>
|   ...
|...
```

In [16]:
import json

Next, I will load the json file as an object. This will make it easy to work with in python.

In [17]:
with open('WLASL/WLASL_v0.3.json', 'r') as file:
    data = json.load(file)

The json file is now loaded as an object in python

Now I will test out how this json module works and how to parse the different properties I need.

The bracket character [] represents a list, and the curly brackets {} represent objects that have name:value pairs.

The json file starts with [], which is a list of objects. Each object has a word and another list for all the instances of that word. Each instance is an object that has name:value pairs for that instance such as: video URL, video id, start time, end time, fps, and more.

The object for the first word can be accessed by accessing the first element of the list in the data json object:

In [18]:
data[0]

{'gloss': 'book',
 'instances': [{'bbox': [385, 37, 885, 720],
   'fps': 25,
   'frame_end': -1,
   'frame_start': 1,
   'instance_id': 0,
   'signer_id': 118,
   'source': 'aslbrick',
   'split': 'train',
   'url': 'http://aslbricks.org/New/ASL-Videos/book.mp4',
   'variation_id': 0,
   'video_id': '69241'},
  {'bbox': [190, 25, 489, 370],
   'fps': 25,
   'frame_end': -1,
   'frame_start': 1,
   'instance_id': 1,
   'signer_id': 90,
   'source': 'aslsignbank',
   'split': 'train',
   'url': 'https://aslsignbank.haskins.yale.edu/dictionary/protected_media/glossvideo/ASL/BO/BOOK-418.mp4',
   'variation_id': 0,
   'video_id': '65225'},
  {'bbox': [262, 1, 652, 480],
   'fps': 25,
   'frame_end': -1,
   'frame_start': 1,
   'instance_id': 2,
   'signer_id': 110,
   'source': 'valencia-asl',
   'split': 'train',
   'url': 'https://www.youtube.com/watch?v=0UsjUE-TXns',
   'variation_id': 0,
   'video_id': '68011'},
  {'bbox': [123, 19, 516, 358],
   'fps': 25,
   'frame_end': 60,
   'frame

The number of words in this object should be 2000. Getting the length of how many dictionary objects are in the first list should result in 2000.

In [19]:
len(data)

2000

This means all the words are in the json object

To access the word, I specify the key of the value I want. In this case, the word is the value for the key called 'gloss':

In [20]:
data[0]['gloss']

'book'

So from this, the first word in the json file is 'book'

The first instance of book is then accessed by appending [0]:

In [21]:
data[0]['instances'][0]

{'bbox': [385, 37, 885, 720],
 'fps': 25,
 'frame_end': -1,
 'frame_start': 1,
 'instance_id': 0,
 'signer_id': 118,
 'source': 'aslbrick',
 'split': 'train',
 'url': 'http://aslbricks.org/New/ASL-Videos/book.mp4',
 'variation_id': 0,
 'video_id': '69241'}

The len() function can be used again here to get how many instances the word 'book' has.

In [22]:
len(data[0]['instances'])

40

The word 'book' has 40 instances.

Finally, to get the video-id to open the correct video, I use the video_id name.

In [23]:
data[0]['instances'][0]['video_id']

'69241'

This means that the video named 69241.mp4 is the video for the first instance of the word 'book'.

The same process can be used to extract the split (train, test, or validation) that this video is a part of, and how many frames are in this video clip.

note: This dataset already has the videos so there is no need to download the video again from the URL. But that would be an extra step if the dataset did not have the videos already.

Now that I know how to access the different name-value pairs, I can begin organizing the videos into train, test, and validation splits. Then, splitting the videos into frames.

### Organizing Videos:

Before getting the frames, I want to organize the videos into their own directories. This will make debugging later easier and it will make looking for the right videos easier.

To copy the video file, I first want to check if I have made the directory where the video will be saved. If a directory has not been made yet, I will create a new one and save the video to it.

To check if a directory exists, I can use the ```os.path.exists``` function. Then to create a directory, I use the ```os.makedirs``` function

In [34]:
import os

In [35]:
if not os.path.exists('./test_makedir/1/1/'):
    print(f'No directory called ./test_makeidr/1/1/. Creating one now')
    os.makedirs('./test_makedir/1/1/')

No directory called ./test_makeidr/1/1/. Creating one now


Now I can put this in a function that I will call for each video

In [36]:
def copy_video(directory_path, video_id):
    if not os.path.exists(directory_path):
        print(f'No directory called {directory_path}. Creating one now')
        os.makedirs(directory_path)

    copy_command = 'cp WLASL/videos/' + video_id + '.mp4' + ' ' + directory_path + '/' + video_id + '.mp4'
    os.system(copy_command)

Next, I want to start organizing the videos. I will go through each instance in each word and get the split type and video id. Then I will find the video with that video id and save it to one of: train directory, test directory, or validation directory.

In [37]:
def organize_words(end_word, start_word=0):
    for word_id in range(start_word, end_word):
        # List of dictionary objects for each word
        word = data[word_id]['gloss']
        train_instance = 0
        test_instance = 0
        val_instance = 0
        for instance in range(0,len(data[word_id]['instances'])):
            video_id = data[word_id]['instances'][instance]['video_id']
            split_type = data[word_id]['instances'][instance]['split']
            match split_type:
                case 'train':
                    directory_path = os.path.join(split_type, str(word_id), str(train_instance))
                    copy_video(directory_path, video_id)
                    train_instance += 1
                case 'test':
                    directory_path = os.path.join(split_type, str(word_id), str(test_instance))
                    copy_video(directory_path, video_id)
                    test_instance += 1
                case 'val':
                    directory_path = os.path.join(split_type, str(word_id), str(val_instance))
                    #if not os.path.exists(directory_path):
                    #    print(f'No directory called {directory_path}. Creating one now')
                    #    os.makedirs(directory_path)
                    #copy_command = 'cp WLASL/videos/' + video_id + '.mp4' + ' ' + directory_path + '/' + video_id + '.mp4'
                    #os.system(copy_command)
                    copy_video(directory_path, video_id)
                    val_instance += 1                       

Now I can call this and specify how many words I want to go through and where to start. For example, I will copy and organize the videos for the first 10 words:

In [38]:
organize_words(10, start_word=0)

No directory called train/0/0. Creating one now
No directory called train/0/1. Creating one now
No directory called train/0/2. Creating one now
No directory called train/0/3. Creating one now
No directory called train/0/4. Creating one now
No directory called val/0/0. Creating one now
No directory called train/0/5. Creating one now
No directory called train/0/6. Creating one now
No directory called train/0/7. Creating one now
No directory called train/0/8. Creating one now
No directory called train/0/9. Creating one now
No directory called train/0/10. Creating one now
No directory called train/0/11. Creating one now
No directory called train/0/12. Creating one now
No directory called train/0/13. Creating one now
No directory called test/0/0. Creating one now
No directory called test/0/1. Creating one now
No directory called train/0/14. Creating one now
No directory called val/0/1. Creating one now
No directory called test/0/2. Creating one now
No directory called val/0/2. Creating one 

Now, the videos for 10 words have been copied and organized into the directory structure I want. Next, I will find each video, and split it into frames

### Cropping Videos

Before doing that though, I found an issue. Some of the videos are not of the word itself, but something about the word. For example, the videos about the word 'book' are not just about book. For example, some are 'history book', 'book photography', etc... However, they all contain the sign for the word 'book' at the end of the video. So I will create a function that will crop these videos to only include the book part of the video.

This doesn't happen for all the words, so I will watch the video and see if the video includes something other than the word itself. If it does, I will use the below function to crop the video to the times that include only that word. This is the only reliable method I have found, but it should not be a problem since the videos are all between 2-4 seconds long. So not much time will be wasted.

In [155]:
import cv2

In [147]:
def crop_video_frames(video_path, start_time=0.0, end_time=0.0):
    cap = cv2.VideoCapture(video_path)
    video_path_parts = video_path.split('/')
    video_id = video_path_parts[3]
    video_dir = video_path_parts[0] + '/' + video_path_parts[1] + '/' + video_path_parts[2]
    video_output = os.path.join(f'{video_dir}', f'{video_id}_cropped.mp4')

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frames_total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if start_time < 0:
        start_frame = int(frames_total + int(start_time * fps))
    else:
        start_frame = int(fps * start_time)
    
    if end_time < 0:
        end_frame = int(frames_total + int(end_time * fps))
    elif end_time == 0:
        end_frame = frames_total
    else:
        end_frame = int(fps * end_time)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    #print(str(start_frame))
    #print(str(end_frame))
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    frame_num = start_frame
    video = cv2.VideoWriter(video_output, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    while True:
        # Read next frame
        ret, frame = cap.read()

        # If no more frames, stop.
        if not ret:
            break

        # increment frame number
        frame_num += 1

        if frame_num < end_frame:
            # Video writer to create mp4 file
            video.write(frame)

    # Release the video file
    video.release()
    cap.release()
    cv2.destroyAllWindows()
    print(f"Video generated successfully to {video_output}. Video is {end_frame - start_frame} frames long")

I will test it on one video that was about english book and grammar. The cropped videos should be the sign 'book' only.

In [154]:
crop_video_frames('train/0/6/07085.mp4', -1.8, -0.2)

Video generated successfully to train/0/6/07085.mp4_cropped.mp4. Video is 40 frames long


After watching the video, I have verified that this works.

I will create three lists: one of videos that need to be cropped, one of the begining time to crop, and one of the end crop time.

Now I will go to save the video frames

### Saving Video Frames

To save the frames, I will navigate through directory tree and find each video. Then I will call the ```save_video_frames``` function to save the frame to a new directory called frames. The new structure will look like: ```<split>/<word_id>/<instance_id>/frames/<instance_id>_<frame_num>.png```

For example, the video for the first instance of the first training word will save the first frame to: train/0/0/frames/0_0.png

To split the video into frames, I will load the video using opencv, and save each frame as a png image. The length in frames will be returned for comparison between videos

I will be passing the whole output path, so I want to extract the instance id to append to the frames file name. To do this, I will split the path and get the 3rd element which should be the instance id

In [44]:
example_string = 'train/0/1/frames'
output_str_split = example_string.split('/')
print(output_str_split)
instance_id = output_str_split[2]
print(instance_id)

['train', '0', '1', 'frames']
1


Now I will create the function to extract and save video frames.

In [119]:
def save_video_frames(video_path, output_path):
    cap = cv2.VideoCapture(video_path)

    if not os.path.exists(output_path):
        print(f'No directory called {output_path}. Creating one now')
        os.makedirs(output_path)

    frame_num = 0

    while True:
        # Read next frame
        ret, frame = cap.read()

        # If no more frames, stop.
        if not ret:
            break

        # increment frame number
        frame_num += 1

        # Save frame
        output_str_split = output_path.split('/')
        instance_id = output_str_split[2]
        frame_path = os.path.join(output_path, f'{instance_id}_{frame_num}.png')
        
        if not os.path.exists(output_path):
            print(f'No directory called {output_path}. Creating one now')
            os.makedirs(output_path)

        cv2.imwrite(frame_path, frame)

        #print(f'Frame {frame_num} has been saved to {frame_path}')
    
    # Release video capture
    cap.release()
    print(f'All {frame_num+1} frames for the video at {video_path} have been successfully saved')
    # Return how many frames make up the video
    return frame_num+1


Testing this with one video:

In [ ]:
save_video_frames('train/0/0/69241.mp4', 'train/0/0/frames')

This works. So I can use this in a loop to do this for all videos organized.

To go through a directory and find each file, I can use the ```os``` module which I already imported before.

In [ ]:
directory = 'train'
for root, dirs, files in os.walk(directory):
    for file_name in files:
        if file_name.endswith('.mp4'):
            #print(file_name, dirs, root)
            #print(os.path.join(root, file_name))
            word_id = root.split('/')[1]
            if int(word_id) >=0 and int(word_id) < 10:
                print(f'{os.path.join(root, file_name)}, word: {data[int(word_id)]["gloss"]}')

This prints the whole path for each video. I can use this to then pass the path to the save_video_frames function and save the frames of each video:

In [160]:
def save_mult_video_frames(split_type='train', start_word=0, end_word=1):
    frames = []
    for root, dirs, files in os.walk(split_type):
        for file_name in files:
            if file_name.endswith('.mp4'):
                #print(file_name, dirs, root)
                #print(os.path.join(root, file_name))
                word_id = root.split('/')[1]
                if int(word_id) >= start_word and int(word_id) < end_word:
                    video_frames = save_video_frames(f'{os.path.join(root, file_name)}', f'{os.path.join(root, "frames")}')
                    frames.append(video_frames)
                    #print(f'{os.path.join(root, file_name)}, word: {data[int(word_id)]["gloss"]}')
    return frames

To save the frames of all training videos of the first word, I can just call the function with default values.

In [161]:
frames_list = save_mult_video_frames(end_word=10)

All 57 frames for the video at train/7/17/63238.mp4 have been successfully saved
All 61 frames for the video at train/7/10/66778.mp4 have been successfully saved
All 33 frames for the video at train/7/7/63227.mp4 have been successfully saved
All 85 frames for the video at train/7/3/63239.mp4 have been successfully saved
All 55 frames for the video at train/7/1/68890.mp4 have been successfully saved
All 42 frames for the video at train/7/15/67066.mp4 have been successfully saved
All 63 frames for the video at train/7/4/63240.mp4 have been successfully saved
All 66 frames for the video at train/7/5/63241.mp4 have been successfully saved
All 36 frames for the video at train/7/8/63228.mp4 have been successfully saved
All 133 frames for the video at train/7/11/63232.mp4 have been successfully saved
All 57 frames for the video at train/7/12/66779.mp4 have been successfully saved
All 55 frames for the video at train/7/0/69534.mp4 have been successfully saved
All 80 frames for the video at tra

In [163]:
import numpy as np

In [169]:
print(f'Frames list of all videos converted: {frames_list}')
print(f'Frames list size: {len(frames_list)}')
num_over_50 = 0
num_bw_40_50 = 0
num_less_40 = 0
for frames in frames_list:
    if frames > 50:
        num_over_50 += 1
    elif frames < 40:
        num_less_40 += 1
    else:
        num_bw_40_50 += 1
print(f'Count of videos with over 50 frames: {num_over_50}. Count of videos less than 40 frames: {num_less_40}. Count of videos between 40 and 50 frames: {num_bw_40_50}')

Frames list of all videos converted: [57, 61, 33, 85, 55, 42, 63, 66, 36, 133, 57, 55, 80, 95, 75, 45, 40, 62, 42, 61, 78, 59, 97, 43, 28, 66, 46, 48, 74, 42, 33, 70, 73, 88, 26, 93, 38, 47, 56, 106, 61, 60, 55, 56, 43, 66, 83, 62, 49, 80, 49, 52, 47, 62, 46, 36, 59, 90, 43, 56, 70, 101, 62, 78, 80, 48, 52, 99, 114, 81, 70, 21, 62, 132, 33, 97, 97, 119, 119, 103, 56, 51, 53, 78, 58, 53, 89, 40, 38, 78, 44, 93, 121, 73, 55, 73, 40, 56, 54, 40, 75, 126, 88, 68, 61, 76, 65, 62, 26, 47, 51, 46, 29, 127, 64, 66, 34, 19, 88, 83, 29, 101, 125, 65, 94, 64, 91, 97, 122, 81, 34, 105, 78, 96, 80, 65, 32, 59, 81, 70, 94, 48, 100, 74, 76, 76, 52, 35, 61, 40, 63, 62, 62, 30, 35, 65, 103, 91, 71, 113, 69, 97, 79, 87, 56, 79, 63, 99, 77, 89, 32, 81, 93, 52, 59, 88, 72, 72, 90, 119, 41, 56, 39, 63, 77, 103, 154, 56, 89, 68, 92, 39, 47, 90, 70, 36, 63, 76, 54, 40]
Frames list size: 200
Count of videos with over 50 frames: 149. Count of videos less than 40 frames: 24. Count of videos between 40 and 50 fr

Next, I will need to pass these frames to mediapipe and generate the landmark points. Then I will save these points as a numpy array to use in training.